# Small Language Workshop Part 1

# Tokenization from Scratch: Building a Byte Pair Encoder (BPE)

In this notebook, we will implement a **byte-level tokenizer** similar to the one used in GPT-style models.

By the end, you will:
- Understand why tokenization is necessary
- Implement Byte Pair Encoding from scratch
- Train a tokenizer on real Unicode text
- Encode and decode text losslessly
- Understand why tokenization matters for Small Language Models

## 1. Why Tokenization?

Neural networks operate on **numbers**, but language is **text**.

Key challenges:
- Variable-length sequences
- Unicode & emojis
- Finite vocabulary
- Efficiency (context length matters!)

💡 **Key idea:** Tokenization is a *compression problem*.

<img src='Neural_Network.png'>

## 2. Text → Bytes

Instead of characters or words, GPT-style models operate on **bytes**.

Why?
- Every string can be represented as bytes
- No unknown tokens
- Unicode-safe

In [ ]:
text = "hello 😄 students"
encoded = text.encode("utf-8")

print("Original text:", text)
print("UTF-8 Encoding:", encoded) # Bytes values
print("List Encoded UTF-8:", list(encoded)) # Decimal Values in Hexadecimal

🧠 **Think**
- Why does the emoji produce multiple numbers?
- Why might this be better than character-level tokenization?

## 3. Initial Vocabulary

We start with a vocabulary of **256 tokens**, one for each possible byte.

In [2]:
vocab = {i: bytes([i]) for i in range(256)}

# Sanity check
print(vocab[97], vocab[97].decode("utf-8"))  # 'a'

# So we can take the previous values of the string "hello 😄 students"
print(vocab[104], bytes([encoded[0]]))  # It should match
print(vocab[240], bytes([encoded[6]]))  # It should match

b'a' a


NameError: name 'encoded' is not defined

## 4. Counting Adjacent Pairs

Byte Pair Encoding works by repeatedly merging the **most frequent adjacent pair**.

In [ ]:
def get_pair(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        # TODO: count how many times each pair appears

    return counts

In [ ]:
ids = [1, 2, 3, 1, 2]
print(get_pair(ids))

# (1,1) or (2,2) does not count because the elements are equal

## 5. Merging Pairs

When we merge a pair `(a, b)` into a new token `k`,
we replace all occurrences of `(a, b)` with `k`.

<img src = 'BPE_Algorithm.png'>

In [ ]:
def merge(ids, pair, index):
    new_ids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and (ids[i], ids[i+1]) == pair:
            new_ids.append(index)
            i += 2
        else:
            new_ids.append(ids[i])
            i += 1
    return new_ids

In [ ]:
merge([1, 2, 3, 1, 2], (1, 2), 256)

What happened to sequence length? Why is this useful?

## 6. Training the Tokenizer

We will now repeatedly:
1. Count all adjacent pairs
2. Select the most frequent one
3. Merge it into a new token

In [ ]:
class BasicTokenizer:
    def __init__(self, vocab_size):
        self.vocab_size = vocab_size
        self.vocabulary = {i: bytes([i]) for i in range(256)}
        self.merges = {}

    def train(self, text, verbose=False):
        assert self.vocab_size > 256

        ids = list(text.encode("utf-8"))
        initial_len = len(ids)

        for i in range(self.vocab_size - 256):
            pairs = get_pair(ids)
            pair = max(pairs, key=pairs.get)
            new_id = 256 + i

            ids = merge(ids, pair, new_id)
            self.merges[pair] = new_id
            self.vocabulary[new_id] = (
                self.vocabulary[pair[0]] + self.vocabulary[pair[1]]
            )

        if verbose:
            print("Compression ratio:", initial_len / len(ids))

## 7. Encoding New Text

At inference time, we **must apply merges in the same order they were learned**.

In [ ]:
def encode(self, text):
    ids = list(text.encode("utf-8"))

    while len(ids) > 1:
        pairs = get_pair(ids)
        pair = min(pairs, key=lambda p: self.merges.get(p, float("inf")))
        if pair not in self.merges:
            break
        ids = merge(ids, pair, self.merges[pair])

    return ids

## 8. Decoding Tokens Back to Text

Tokenization must be reversible.

In [ ]:
def decode(self, ids):
        byte_string = b"".join(self.vocabulary[i] for i in ids)
        return byte_string.decode("utf-8")

## 9. Let's Practice!

In [ ]:
class BasicTokenizer:
    def __init__(self, vocab_size):
        self.vocab_size = vocab_size
        self.vocabulary = {i: bytes([i]) for i in range(256)}
        self.merges = {}

    def train(self, text, verbose=False):
        assert self.vocab_size > 256

        ids = list(text.encode("utf-8"))
        initial_len = len(ids)

        for i in range(self.vocab_size - 256):
            pairs = get_pair(ids)
            pair = max(pairs, key=pairs.get)
            new_id = 256 + i

            ids = merge(ids, pair, new_id)
            self.merges[pair] = new_id
            self.vocabulary[new_id] = (
                self.vocabulary[pair[0]] + self.vocabulary[pair[1]]
            )
            #print(sorted( [(v, k) for k,v in pairs.items()], reverse = True) [:10])


        if verbose:
            print("Compression ratio:", initial_len / len(ids))

    def encode(self, text):
        ids = list(text.encode("utf-8"))

        while len(ids) > 1:
            pairs = get_pair(ids)
            pair = min(pairs, key=lambda p: self.merges.get(p, float("inf")))
            if pair not in self.merges:
                break
            ids = merge(ids, pair, self.merges[pair])

        return ids

    def decode(self, ids):
        byte_string = b"".join(self.vocabulary[i] for i in ids)
        return byte_string.decode("utf-8")

In [ ]:
tokenizer = BasicTokenizer(266)
print("The text to encode is the following:", text)
tokenizer.train(text, verbose=True)

encoded = tokenizer.encode("hello 😄 students")
decoded = tokenizer.decode(encoded)

print(encoded)
print(decoded)

In [ ]:
tokenizer = BasicTokenizer(266)
textTest = "hello everyone"
print("The text to encode is the following:", textTest)
tokenizer.train(textTest, verbose=True)

encoded = tokenizer.encode(textTest)
decoded = tokenizer.decode(encoded)

print(encoded)
print(decoded)

## 10. Now , your turn!

Try the following:

1. Change `vocab_size` and measure compression
2. Train on:
   - English text
   - Code
   - Emojis
3. Print the first 20 learned merges
4. Compare with character-level tokenization

✍️ Write short answers below each experiment.

In [ ]:
tokenizer = BasicTokenizer(280)
textTest = "✍️ Write short answers below each experiment."
print("The text to encode is the following:", textTest)
tokenizer.train(textTest, verbose=True)

encoded = tokenizer.encode(textTest)
decoded = tokenizer.decode(encoded)

print(encoded)
print(decoded)

## 10. Why This Matters for Language Models

- Tokens → embeddings
- Fewer tokens → longer context
- Tokenization is the **first inductive bias** of a Transformer
- Good tokenization matters more when the model is small.

This tokenizer can now be plugged directly into a GPT training loop.

<a href="https://tiktokenizer.vercel.app/" >
    <img src="Tiktokenizer.png" >
</a>

# Regex Tokenizer

### 1. Why RegexTokenizer Exists

Your current basicTokenizer trains BPE on raw byte streams:

```
text → bytes → BPE merges
```

That works — but it has drawbacks:

- BPE may merge across semantic boundaries

- Punctuation, whitespace, and numbers get mixed

- Training is slower and noisier

- Tokens can become syntactically awkward

RegexTokenizer fixes this by adding structure before BPE.

### 2️. Core Idea (One Sentence)

RegexTokenizer first splits text into meaningful chunks using regex, then applies BPE inside each chunk independently.

This is exactly how GPT-2 / GPT-3 style tokenizers work.

### 3. High-Level Pipeline

```
Text
 ↓
Regex split (words, numbers, punctuation, spaces)
 ↓
Each chunk → UTF-8 bytes
 ↓
Byte Pair Encoding (BPE)
 ↓
Final token IDs
```

So instead of training BPE on everything, we train it on pre-segmented text units.

Let’s decode what it does.

What the Regex Captures
Pattern	Meaning

| Pattern    | Meaning |
| -------- | ------- |
| \p{L}+  | Letters (words)    |
| \p{N}+ | Numbers     |
| [^ \s\p{L}\p{N}]+    | Punctuation / symbols    |
| \s+    | Whitespace    |
| 's, 't, etc.    | English contractions   |

In [ ]:
import regex as re

pattern = re.compile(
    r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,2}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+""", # NOTE: this split pattern deviates from GPT-4 in that it is used \p{N}{1,2} instead of \p{N}{1,3}
# I did this because I didn't want to "waste" too many tokens on numbers for smaller vocab sizes.
# I haven't validated that this is actually a good idea, TODO.
    re.UNICODE
)

In [ ]:
text = "Hello 😄 students"
print(text)
print("-"*50)
print("Splitting Regex Pattern Capture:\n", re.findall(pattern, text))

#### ! Important: Spaces are kept, not discarded.

In [ ]:
text = "Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception."
print(text)
print("-"*50)
re.findall(pattern, text)

### 3. Why This Matters Before BPE
Without Regex (Your Basic Tokenizer)

BPE might merge:

```

"o " + "t" → "o t"

```

Which is meaningless.

With RegexTokenizer

BPE operates inside units like:

- "hello"

- " there"

- "!"

This ensures:

- Cleaner merges

- Faster convergence

- More interpretable tokens


In [ ]:
sample = "Hello there! I'm 26. 😄\nNew line."
chunks = re.findall(pattern, sample)
chunks

In [ ]:
vocabulary = {i: bytes([i]) for i in range(256)}
print(vocabulary[97], vocabulary[97].decode("utf-8"))  # 'a'
print(list("😄".encode("utf-8")))

### BPE primitives: count adjacent pairs + merge

BPE repeatedly merges the most frequent adjacent pair into a new token id.


In [ ]:
def get_pair(ids, counts):
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, index):
    '''
    This function will iterate over ids and every time
    it sees a instance of pair, it will take that pair
    and instead put index , then it will return the list
    merge()
    list = [1,2, 3, 4, 1, 2]
    merge(list, (1,2). 257)
    list = [257, 3, 4, 257, 3]
    '''

    new_ids = []
    i = 0
    while i < len(ids):
        if i <len(ids) - 1 and  (ids[i], ids[i+1]) == pair:
            new_ids.append(index)
            i += 2
        else:
            new_ids.append(ids[i])
            i += 1
    return new_ids

### 4. RegexTokenizer overview

Training:
1. Split text into regex chunks
2. Convert each chunk to UTF-8 bytes (list of ints)
3. Count pairs across **all chunks**
4. Merge the most frequent pair across **all chunks**
5. Repeat until reaching vocab_size

Encoding:
1. Split input text into regex chunks
2. Encode each chunk with learned merges
3. Concatenate token ids

Decoding:
- Map ids → bytes → UTF-8 string

In [ ]:
import pickle

In [ ]:

class regexTokenizer:

    def __init__(self, vocab_size, pattern):

        self.vocab_size = vocab_size
        self.vocabulary = {i : bytes([i]) for i in range(256)}
        self.merges = {}
        self.pattern = re.compile(pattern)

    def train(self, text, verbose = False):
        # Encode the text
        # Iterate over text, self.vocab_size - 256 times
        # count all of the pairs in a dictionary
        # choose the pair with the highest frequency
        # merge that pair as a new token
        # add that token to the vocab
        # {256: byte_string}
        # add to self.merges = {byte_string: 256}

        assert self.vocab_size > 256
        number_merges = self.vocab_size - 256

        text_chunks = re.findall(self.pattern, text)
        encoded_chunks = [list(text_chunk.encode('utf-8')) for text_chunk in text_chunks]

        length_initial = sum([len(encoded_chunk) for encoded_chunk in encoded_chunks])

        for i in range(number_merges):
            pairs = {}
            for encoded_chunk in encoded_chunks:
                pairs = get_pair(encoded_chunk, pairs)

            pair = max(pairs, key = pairs.get)
            index = 256 + i
            encoded_chunks = [merge(encoded_chunk,pair,index) for encoded_chunk in encoded_chunks]
            self.merges[pair] = index
            self.vocabulary[index] = self.vocabulary[pair[0]] + self.vocabulary[pair[1]]
            #print(sorted( [(v, k) for k,v in pairs.items()], reverse = True) [:10])

        if verbose:
            length_final = sum([len(encoded_chunk) for encoded_chunk in encoded_chunks])
            compression = length_initial/length_final
            print(length_initial, length_final)
            print(compression)

    def encode(self, text):

        text_chunks = re.findall(self.pattern, text)
        encoded_text = []

        for text_chunk in text_chunks:
            encoded_chunk = self.encode_chunk(text_chunk)
            encoded_text.extend(encoded_chunk)
        return encoded_text

    def encode_chunk(self, text):
        '''
        self.merges is important here

        we get text, and then we convert that text to byte strings, then to integers
        and then we iterate over the text until all pairs of
        merges that are possible under the trained tokenizer
        have been completed

        '''

        ids = list(text.encode('utf-8'))

        for pair, index in self.merges.items():
            ids = merge(ids, pair, index)

        return ids

    def decode(self, ids):
        '''
        decode gets ids
        1. convert the ids to their byte strings
        2. convert the byte strings to strings via the vocabulary
        3. then return the decoded_text
        '''

        byte_strings = b''.join([bytes(self.vocabulary[i]) for i in ids])
        decoded_text =  byte_strings.decode('utf-8')
        return decoded_text

    def save(self, path):
        with open(path, "wb") as file:
            pickle.dump(
                {
                    "merges": self.merges,
                    "vocabulary": self.vocabulary,
                    "pattern": self.pattern
                },
                file
            )

    @classmethod
    def load(cls, path):
        tokenizer = cls(300, pattern)

        with open(path , "rb") as file:
            data = pickle.load(file)
            tokenizer.merges = data["merges"]
            tokenizer.vocabulary = data["vocabulary"]
            tokenizer.pattern = data["pattern"]
        return tokenizer


In [ ]:
tokenizer = regexTokenizer(300, pattern)

In [ ]:
text = "Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception."
tokenizer.train(text, True)

### 5. Exercise (10 minutes)
1. Chunk inspection:
   - Print chunks for:
   - English sentence
   - Code snippet
   - Emoji-heavy text
2. Compression study:
   - Train with vocab_size in {300, 500, 1000}
   - Report compression ratio
3. Token boundary study:
   - Compare tokenization for:
   - "hello there"
   - "hello there" (double space)
   - "hello\nthere"
4. Compare the tokenization between the basic Tokenizer and regex Tokenizer"

In [ ]:
# TO DO

# Nanochat Tokenizer

We move from educational tokenizers to the **final tokenizer**
used in NanoChat.

You will learn:
- How GPT-style tokenizers handle *chat*
- What special tokens are and why they matter
- How conversations are rendered into `(input_ids, loss_mask)`
- How this enables supervised fine-tuning (SFT)

In [ ]:
# First we need to download the weights https://huggingface.co/karpathy/nanochat-d32/tree/main
# Put the tokenizer.pkl in ~/.cache/nanochat/tokenizer directory

In [ ]:
!git clone https://github.com/karpathy/nanochat.git


In [ ]:
%cd nanochat

In [ ]:
# Remove ipykernel and add
#[tool.setuptools.packages.find]
#where = ["."]
#include = ["nanochat*"]
#exclude = ["class*", "dev*"]
# to pyproject.toml
!pip install -e .

In [ ]:
from nanochat.tokenizer import get_tokenizer

In [ ]:
%cd /root

In [ ]:
%mkdir /root/.cache/nanochat/tokenizer/

In [ ]:
%cd /root/.cache/nanochat/tokenizer

In [ ]:
from huggingface_hub import hf_hub_download
# Download the specific file and get its local file path
tokenizer_pkl_path = hf_hub_download(repo_id="karpathy/nanochat-d34", filename="tokenizer.pkl")
tokenizer_pt_path = hf_hub_download(repo_id="karpathy/nanochat-d34", filename="token_bytes.pt")

In [ ]:
tokenizer_pkl_path

In [ ]:
%cp '/root/.cache/huggingface/hub/models--karpathy--nanochat-d34/snapshots/c48357d43863a3a6cdc5f5db5b4ec5964e4192d6/tokenizer.pkl' '/root/.cache/nanochat/tokenizer/tokenizer.pkl'

In [ ]:
%cd /content/nanochat/
%ls

In [ ]:
tokenizer = get_tokenizer()
print(type(tokenizer))

### 1. Special Tokens

Chat models need *control tokens* to:
- separate user vs assistant
- delimit messages
- support tools (python blocks, outputs)

In [ ]:
special_tokens = tokenizer.get_special_tokens()
special_tokens

In [ ]:
# Lets encode these special tokens
for tok in special_tokens:
    print(f"{tok:25s} → id {tokenizer.encode_special(tok)}")

In [ ]:
# Lets decode it again to see if everything works well
encoded_special_tokens = [ tokenizer.encode_special(tok) for tok in list(special_tokens) ]
for id in encoded_special_tokens:
    print(f"{id} → id {tokenizer.decode([id])}")

In [ ]:
# Lets try with the text we already worked
text = "Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception."
ids = tokenizer.encode(text)
decoded = tokenizer.decode(ids)

ids, decoded

In [ ]:
conversation = {
    "messages": [
        {"role": "user", "content": "What is a transformer?"},
        {"role": "assistant", "content": "A transformer is a neural network based on attention."}
    ]
}

In [ ]:
ids, loss_mask = tokenizer.render_conversation(conversation)

print("Number of tokens:", len(ids))
print("Loss tokens:", sum(loss_mask))

In [ ]:
def printUserAssistantType(mask):
    if mask == 0:
        return "User"
    else:
        return "Assistant"

In [ ]:
decoded_tokens = [tokenizer.decode([i]) for i in ids]

for t, m in zip(decoded_tokens, loss_mask):
    print(f"{repr(t):20s}  mask={m} {printUserAssistantType(m)}")

### 2. Why Masking?

Without masking:
- model would be trained to predict user prompts
- learning becomes unstable
- chat behavior degrades

Masking enforces:
P(assistant | user)

### 3. Exercise (15 minutes)
1. Create a 3-turn conversation
2. Render it
3. Count how many tokens are supervised
4. Inspect where assistant supervision starts

In [ ]:
## TO DO

## Next Step

We now have:
- Token IDs
- Loss masks

Next:
➡ Let's understand GPT